In [3]:
from joblib import load
import numpy as np
import pandas as pd
import spacy
import re
from fractions import Fraction
from decimal import Decimal, InvalidOperation
import unicodedata
import sys
sys.path.append("..")
from src.data import data_cleaning_util
from src.features import create_features
import unidecode
from collections import Counter
import operator
%pprint

Pretty printing has been turned ON


In [4]:
# Load our trained CRF model and the spacy NLP Model
crf = load('../models/crf_model.joblib')
nlp = spacy.load('en_core_web_lg', disable=['ner','textcat'])

In [5]:
epi_raw = pd.read_json("../data/raw/recipes_raw_epi.json")

In [6]:
epi_raw.head()

,http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,http://www.epicurious.com/recipes/food/views/-an-apple-a-day-51133430,http://www.epicurious.com/recipes/food/views/-blanketed-eggplant-305,http://www.epicurious.com/recipes/food/views/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813,http://www.epicurious.com/recipes/food/views/-bow-tie-pasta-with-zucchini-101932,http://www.epicurious.com/recipes/food/views/-burnt-carrots-and-parsnips-56390131,http://www.epicurious.com/recipes/food/views/-california-roll-salad-12246,http://www.epicurious.com/recipes/food/views/-candy-corn-frozen-citrus-cream-pops-368770,http://www.epicurious.com/recipes/food/views/-candy-corn-pumpkin-blondies-51254510,http://www.epicurious.com/recipes/food/views/-cannoli-ice-cream-sandwiches-242004,...,http://www.epicurious.com/recipes/food/views/zucchini-wrapped-red-snapper-with-tomato-cumin-and-orange-sauce-10045,http://www.epicurious.com/recipes/food/views/zuni-ricotta-gnocchi-241532,http://www.epicurious.com/recipes/food/views/zuni-roast-chicken-with-bread-salad-56389456,http://www.epicurious.com/recipes/food/views/zuni-rolls-with-raspberry-chipotle-sauce-15259,http://www.epicurious.com/recipes/food/views/zuppa-di-cavolo-nero-cannellini-e-salsicce-kale-white-bean-and-sausage-soup-363386,http://www.epicurious.com/recipes/food/views/zuppa-inglese-394,http://www.epicurious.com/recipes/food/views/zwetschgenkuchen-14600,http://www.epicurious.com/recipes/seared-scallops-with-tomato-water-lime-and-mint-51242060-recipe,http://www.epicurious.com/simple-syrup-368889-recipe,http://www.epicurious.com/suzanne-goin-s-corned-beef-and-cabbage-with-parsley-mustard-sauce-56389323-recipe
avg_rating,2.5,3.5,3,4,3.5,3.5,3.5,2,3,3,...,3,3,4,3.5,3.5,4,3,None,None,None
best_rating,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,None,None,None
ingredients,"[2 or 3 large garlic cloves, a 2-ounce jar dic...","[1/4 cup (1/2 stick) unsalted butter, 4-5 medi...","[8 small Japanese eggplants, peeled, 16 large ...","[1 lemon, zested, juiced, 1/2 shallot, finely ...","[6 small zucchini, 2 teaspoons salt, 2 cups pa...","[1 1/2 pounds carrots, peeled, halved lengthwi...","[1 1/2 cups long-grain rice, 1/4 cup plus 3 ta...","[2 (14-ounce) cans sweetened condensed milk, 1...","[1 cup (2 sticks) cold unsalted butter, plus m...","[3/4 cup whole-milk ricotta, 1/4 cup cream che...",...,"[2 teaspoons cumin seeds, four 6- to 7-ounce r...","[1 pound fresh ricotta (2 cups), 2 large cold ...","[One small chicken, 2 3/4 to 3 1/2 pounds, 4 t...",[1 cup fresh raspberries or frozen unsweetened...,"[2 Italian sausages, skins removed and meat cr...","[4 cups milk (do not use low-fat or nonfat), 1...","[1 cup unbleached all-purpose flour, dash of s...",[],[],[]
instructions,Force garlic through a garlic press into a lar...,Melt butter in a large heavy skillet over medi...,Place eggplants on double thickness of paper t...,"Combine lemon zest, lemon juice, shallot, toma...",Cut zucchini crosswise into 1/8-inch-thick sli...,"Preheat oven to 450°F. Toss carrots, parsnips,...",Into a large saucepan of salted boiling water ...,"Put one can of milk in each of 2 bowls, then w...",Preheat oven to 350°F. Lightly butter a 9- by ...,Stir together all ingredients except chocolate...,...,Preheat oven to 450°F.\nIn a small dry heavy s...,Check the cheese for wetness. If you are lucky...,Remove and discard the lump of fat inside the ...,In a small saucepan combine sauce ingredients ...,Sauté the crumbled sausage in the oil until br...,Bring milk and 1/2 cup plus 3 tablespoons suga...,"To make the crust using a food processor, fitt...",,,
num_reviews,25,3,4,2,48,2,83,1,15,1,...,8,14,4,47,14,7,6,None,None,None


In [7]:
# We need to transpose the matrix so that we have a the recipes as rows and ingredients as columns
epi_data = epi_raw.transpose()

# Select a subset so processing happend faster
#epi_data = epi_data.head(1000)
epi_data.head()

,avg_rating,best_rating,ingredients,instructions,num_reviews,prepare_again_rating,tags,title,total_time,worst_rating,yields
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s)
http://www.epicurious.com/recipes/food/views/-an-apple-a-day-51133430,3.5,4,"[1/4 cup (1/2 stick) unsalted butter, 4-5 medi...",Melt butter in a large heavy skillet over medi...,3,0.67,"[Ginger, Dessert, Bake, Apple, Almond, Fall, R...","""An Apple a Day""",0,0,6 item(s)
http://www.epicurious.com/recipes/food/views/-blanketed-eggplant-305,3,4,"[8 small Japanese eggplants, peeled, 16 large ...",Place eggplants on double thickness of paper t...,4,0.75,"[Tomato, Vegetable, Appetizer, Side, Vegetaria...","""Blanketed"" Eggplant",0,0,4 serving(s)
http://www.epicurious.com/recipes/food/views/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813,4,4,"[1 lemon, zested, juiced, 1/2 shallot, finely ...","Combine lemon zest, lemon juice, shallot, toma...",2,1,"[Condiment/Spread, Tomato, Appetizer, Kid-Frie...","""Bloody Mary"" Tomato Toast with Celery and Hor...",0,0,6 serving(s)
http://www.epicurious.com/recipes/food/views/-bow-tie-pasta-with-zucchini-101932,3.5,4,"[6 small zucchini, 2 teaspoons salt, 2 cups pa...",Cut zucchini crosswise into 1/8-inch-thick sli...,48,0.93,"[Pasta, Vegetarian, Quick & Easy, Dinner, Basi...","""Bow-Tie"" Pasta with Zucchini",0,0,4 serving(s)


In [8]:
c = Counter()

epi_data["ingredients"].apply(lambda x: c.update(list(str(x))))

# We expect alphabet, digit, and punctuation characters so lets remove them
for i in list(c):
    if i.isalpha() or i.isdigit():
        del c[i] 
sorted(c.keys())

[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '_',
 '{',
 '|',
 '}',
 '¤',
 '§',
 '®',
 '°',
 '¼',
 '½',
 '¾',
 '¿',
 '×',
 '́',
 '‐',
 '‑',
 '–',
 '—',
 '’',
 '‚',
 '“',
 '”',
 '‟',
 '•',
 '‧',
 '‱',
 '″',
 '‿',
 '⁄',
 '™',
 '⅓',
 '⅔',
 '⅛',
 '⅞',
 '�']

In [9]:
# Drop rows with empty ingredients
num_rows = epi_data.shape[0]
epi_data = epi_data[epi_data.astype(str)['ingredients'] != '[]']
print("Dropped {} rows with no ingredients".format(num_rows - epi_data.shape[0]))

Dropped 102 rows with no ingredients


In [10]:
# We do two things next, we create a new dataframe to contain all the ingredient info
# and rearrange it so that it has one ingredient per row. All the non-ingredient columns
# are kept in the original dataframe.
ingredients = epi_data['ingredients'].apply(pd.Series, 1).stack()
ingredients.index = ingredients.index.droplevel(-1)
ingredients.name = 'input'
epi_ingredients = epi_data.join(ingredients)
epi_ingredients.head()

,avg_rating,best_rating,ingredients,instructions,num_reviews,prepare_again_rating,tags,title,total_time,worst_rating,yields,input
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s),2 or 3 large garlic cloves
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s),a 2-ounce jar diced pimientos
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s),3 cups coarsely grated sharp Cheddar (preferab...
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s),1/3 to 1/2 cup mayonnaise
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,"[2 or 3 large garlic cloves, a 2-ounce jar dic...",Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0,2 item(s),crackers


In [11]:
# Drop non ingredient/yield columns from ingredient dataframe
epi_ingredients = epi_ingredients[["input","yields"]]
epi_ingredients.head()

,input,yields
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2 or 3 large garlic cloves,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,a 2-ounce jar diced pimientos,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,3 cups coarsely grated sharp Cheddar (preferab...,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,1/3 to 1/2 cup mayonnaise,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,crackers,2 item(s)


In [12]:
 # Drop ingredient/yield columns from data dataframe
del epi_data["ingredients"]
del epi_data["yields"]
epi_data.head()

,avg_rating,best_rating,instructions,num_reviews,prepare_again_rating,tags,title,total_time,worst_rating
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.5,4,Force garlic through a garlic press into a lar...,25,0.85,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese",0,0
http://www.epicurious.com/recipes/food/views/-an-apple-a-day-51133430,3.5,4,Melt butter in a large heavy skillet over medi...,3,0.67,"[Ginger, Dessert, Bake, Apple, Almond, Fall, R...","""An Apple a Day""",0,0
http://www.epicurious.com/recipes/food/views/-blanketed-eggplant-305,3,4,Place eggplants on double thickness of paper t...,4,0.75,"[Tomato, Vegetable, Appetizer, Side, Vegetaria...","""Blanketed"" Eggplant",0,0
http://www.epicurious.com/recipes/food/views/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813,4,4,"Combine lemon zest, lemon juice, shallot, toma...",2,1,"[Condiment/Spread, Tomato, Appetizer, Kid-Frie...","""Bloody Mary"" Tomato Toast with Celery and Hor...",0,0
http://www.epicurious.com/recipes/food/views/-bow-tie-pasta-with-zucchini-101932,3.5,4,Cut zucchini crosswise into 1/8-inch-thick sli...,48,0.93,"[Pasta, Vegetarian, Quick & Easy, Dinner, Basi...","""Bow-Tie"" Pasta with Zucchini",0,0


In [13]:
epi_ingredients["input"] = epi_ingredients["input"].apply(data_cleaning_util.clean_epi_html)


In [14]:
# Drop rows that have no input
print(epi_ingredients.shape)
epi_ingredients.dropna(axis=0, subset=["input"], inplace=True)
print(epi_ingredients.shape)
#epi_ingredients["input"] = epi_ingredients["input"].apply(data_cleaning_util.fix_characters)

(351441, 2)
(351441, 2)


In [15]:
epi_ingredients.to_csv("../data/interim/epi_partial_clean.csv")

In [16]:
c = Counter()

epi_ingredients["input"].apply(lambda x: c.update(list(str(x))))

# We expect alphabet, digit, and punctuation characters so lets remove them
for i in list(c):
    if i.isalpha() or i.isdigit():
        del c[i] 
sorted(c.keys())

[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '_',
 '{',
 '|',
 '}',
 '¤',
 '§',
 '\xad',
 '®',
 '°',
 '¼',
 '½',
 '¾',
 '¿',
 '×',
 '́',
 '‐',
 '‑',
 '–',
 '—',
 '’',
 '‚',
 '“',
 '”',
 '‟',
 '•',
 '‧',
 '‱',
 '″',
 '‿',
 '⁄',
 '™',
 '⅓',
 '⅔',
 '⅛',
 '⅞',
 '�']

In [17]:

        
def process_data(input_data):
    # Remove random HTML tags
    
    input_data["input"] = input_data["input"].apply(data_cleaning_util.fix_characters)
    input_data["input"] = input_data["input"].apply(data_cleaning_util.fix_spelling)
    
    input_data["input"] = input_data["input"].apply(data_cleaning_util.clean_epi_html)
    
    

    # Drop rows that have no input
    print(input_data.shape)
    input_data.dropna(axis=0, subset=["input"], inplace=True)
    print(input_data.shape)

    input_data["input"] = input_data["input"].apply(data_cleaning_util.fix_epi_abbreviations)
    input_data["input"] = input_data["input"].apply(data_cleaning_util.fix_numeric_words)
    # Unicode has numerous characters to represent fractions like ¾, we remove these
    input_data["input"] = input_data["input"].apply(data_cleaning_util.clean_epi_unicode_fractions)
    input_data["input"] = input_data["input"].apply(str).apply(unidecode.unidecode)

    # Many ingredient quantities are written as 1 1/2 to represent 1.5
    # The quantity label however is always written as 1.5 so we need to
    # convert these fractions so that the crf can match it
    input_data["input"] = input_data["input"].apply(data_cleaning_util.merge_fractions)

    input_data["input"] = input_data["input"].apply(data_cleaning_util.merge_quantities)

    return input_data

In [18]:
# Clean the ingredients
epi_ingredients = process_data(epi_ingredients)
epi_ingredients.head()

(351441, 2)
(351441, 2)


,input,yields
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,2.50 large garlic cloves,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,a 2 - ounce jar diced pimientos,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,3 cups coarsely grated sharp Cheddar (preferab...,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,0.42 cup mayonnaise,2 item(s)
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,crackers,2 item(s)


In [19]:
def word2features(sent, i):

    features = {
        "bias": 1.0,
        "lemma": sent[i].lemma_,
        "pos": sent[i].pos_,
        "tag": sent[i].tag_,
        "dep": sent[i].dep_,
        "shape": sent[i].shape_,
        "is_alpha": sent[i].is_alpha,
        "is_stop": sent[i].is_stop,
        "is_title": sent[i].is_title,
        "like_num": sent[i].like_num,
        "is__left_punct": sent[i].is_left_punct,
        "is__right_punct": sent[i].is_right_punct,
    }
    if i > 0:
        features.update(
            {
                "-1:lemma": sent[i - 1].lemma_,
                "-1:pos": sent[i - 1].pos_,
                "-1:tag": sent[i - 1].tag_,
                "-1:dep": sent[i - 1].dep_,
                "-1:shape": sent[i - 1].shape_,
                "-1:is_alpha": sent[i - 1].is_alpha,
                "-1:is_stop": sent[i - 1].is_stop,
                "-1:is_title": sent[i - 1].is_title,
                "-1:like_num": sent[i - 1].like_num,
                "-1:is_left_punct": sent[i - 1].is_left_punct,
                "-1:is_right_punct": sent[i - 1].is_right_punct,
            }
        )
        if i > 1:
            features.update(
                {
                    "-2:lemma": sent[i - 2].lemma_,
                    "-2:pos": sent[i - 2].pos_,
                    "-2:tag": sent[i - 2].tag_,
                    "-2:dep": sent[i - 2].dep_,
                    "-2:shape": sent[i - 2].shape_,
                    "-2:is_alpha": sent[i - 2].is_alpha,
                    "-2:is_stop": sent[i - 2].is_stop,
                    "-2:is_title": sent[i - 2].is_title,
                    "-2:like_num": sent[i - 2].like_num,
                    "-2:is_left_punct": sent[i - 2].is_left_punct,
                    "-2:is_right_punct": sent[i - 2].is_right_punct,
                }
            )
    else:
        features["BOS"] = True

    if i < len(sent) - 1:
        features.update(
            {
                "+1:lemma": sent[i + 1].lemma_,
                "+1:pos": sent[i + 1].pos_,
                "+1:tag": sent[i + 1].tag_,
                "+1:dep": sent[i + 1].dep_,
                "+1:shape": sent[i + 1].shape_,
                "+1:is_alpha": sent[i + 1].is_alpha,
                "+1:is_stop": sent[i + 1].is_stop,
                "+1:is_title": sent[i + 1].is_title,
                "+1:like_num": sent[i + 1].like_num,
                "+1:is_left_punct": sent[i + 1].is_left_punct,
                "+1:is_right_punct": sent[i + 1].is_right_punct,
            }
        )
        if i < len(sent) - 2:
            features.update(
                {
                    "+2:lemma": sent[i + 2].lemma_,
                    "+2:pos": sent[i + 2].pos_,
                    "+2:tag": sent[i + 2].tag_,
                    "+2:dep": sent[i + 2].dep_,
                    "+2:shape": sent[i + 2].shape_,
                    "+2:is_alpha": sent[i + 2].is_alpha,
                    "+2:is_stop": sent[i + 2].is_stop,
                    "+2:is_title": sent[i + 2].is_title,
                    "+2:like_num": sent[i + 2].like_num,
                    "+2:is_right_punct": sent[i + 2].is_left_punct,
                    "+2:is_right_punct": sent[i + 2].is_right_punct,
                }
            )
    else:
        features["EOS"] = True

    return features

In [20]:
# have spacy parse the input string with the full pipeline to generate features this will take some time
epi_ingredients["input"] = list(
        nlp.pipe(epi_ingredients["input"].astype("unicode").values, batch_size=50)
    )


# Create our features dict
epi_ingredients["features"] = epi_ingredients["input"].apply(
    lambda doc: [word2features(doc, i) for i in range(len(doc))]
)
epi_ingredients.head()

,input,yields,features
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(2.50, large, garlic, cloves)",2 item(s),"[{'bias': 1.0, 'lemma': '2.50', 'pos': 'NUM', ..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(a, 2, -, ounce, jar, diced, pimientos)",2 item(s),"[{'bias': 1.0, 'lemma': 'a', 'pos': 'DET', 'ta..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(3, cups, coarsely, grated, sharp, Cheddar, (,...",2 item(s),"[{'bias': 1.0, 'lemma': '3', 'pos': 'X', 'tag'..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(0.42, cup, mayonnaise)",2 item(s),"[{'bias': 1.0, 'lemma': '0.42', 'pos': 'NUM', ..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,(crackers),2 item(s),"[{'bias': 1.0, 'lemma': 'cracker', 'pos': 'NOU..."


In [21]:
epi_ingredients.iloc[1]

input                 (a, 2, -, ounce, jar, diced, pimientos)
yields                                              2 item(s)
features    [{'bias': 1.0, 'lemma': 'a', 'pos': 'DET', 'ta...
Name: http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852, dtype: object

In [22]:
epi_ingredients["predicted"] = crf.predict(epi_ingredients["features"].values)
epi_ingredients.head()

,input,yields,features,predicted
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(2.50, large, garlic, cloves)",2 item(s),"[{'bias': 1.0, 'lemma': '2.50', 'pos': 'NUM', ...","[B-QTY, B-OTHER, B-NAME, I-NAME]"
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(a, 2, -, ounce, jar, diced, pimientos)",2 item(s),"[{'bias': 1.0, 'lemma': 'a', 'pos': 'DET', 'ta...","[B-OTHER, B-QTY, B-OTHER, B-UNIT, B-OTHER, B-N..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(3, cups, coarsely, grated, sharp, Cheddar, (,...",2 item(s),"[{'bias': 1.0, 'lemma': '3', 'pos': 'X', 'tag'...","[B-QTY, B-UNIT, B-COMMENT, I-COMMENT, I-COMMEN..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"(0.42, cup, mayonnaise)",2 item(s),"[{'bias': 1.0, 'lemma': '0.42', 'pos': 'NUM', ...","[B-OTHER, B-UNIT, B-NAME]"
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,(crackers),2 item(s),"[{'bias': 1.0, 'lemma': 'cracker', 'pos': 'NOU...",[B-NAME]


In [23]:
epi_ingredients["input"] = epi_ingredients["input"].apply(lambda doc: [token.lemma_ for token in doc])
epi_ingredients.head()

,input,yields,features,predicted
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"[2.50, large, garlic, clove]",2 item(s),"[{'bias': 1.0, 'lemma': '2.50', 'pos': 'NUM', ...","[B-QTY, B-OTHER, B-NAME, I-NAME]"
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"[a, 2, -, ounce, jar, dice, pimiento]",2 item(s),"[{'bias': 1.0, 'lemma': 'a', 'pos': 'DET', 'ta...","[B-OTHER, B-QTY, B-OTHER, B-UNIT, B-OTHER, B-N..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"[3, cup, coarsely, grated, sharp, Cheddar, (, ...",2 item(s),"[{'bias': 1.0, 'lemma': '3', 'pos': 'X', 'tag'...","[B-QTY, B-UNIT, B-COMMENT, I-COMMENT, I-COMMEN..."
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,"[0.42, cup, mayonnaise]",2 item(s),"[{'bias': 1.0, 'lemma': '0.42', 'pos': 'NUM', ...","[B-OTHER, B-UNIT, B-NAME]"
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,[cracker],2 item(s),"[{'bias': 1.0, 'lemma': 'cracker', 'pos': 'NOU...",[B-NAME]


In [24]:
def smartJoin(words):
    """
    Joins list of words with spaces, but is smart about not adding spaces
    before commas.
    """

    input = " ".join(words)

    # replace " , " with ", "
    input = input.replace(" , ", ", ")

    # replace " ( " with " ("
    input = input.replace("( ", "(")

    # replace " ) " with ") "
    input = input.replace(" )", ")")

    return input


def format_ingredient_output(tokens, tags, display=False):
    """Formats the tagger output into a more convenient dictionary"""
    data = [{}]
    display = [[]]
    prevTag = None
    
    for token, tag in zip(tokens, tags):
    # turn B-NAME/123 back into "name"
        tag = re.sub(r'^[BI]\-', "", tag).lower()
        # ---- DISPLAY ----
        # build a structure which groups each token by its tag, so we can
        # rebuild the original display name later.

        if prevTag != tag:
            display[-1].append((tag, [token]))
            prevTag = tag
        else:
            display[-1][-1][1].append(token)
            #               ^- token
            #            ^---- tag
            #        ^-------- ingredient

            # ---- DATA ----
            # build a dict grouping tokens by their tag

            # initialize this attribute if this is the first token of its kind
        if tag not in data[-1]:
            data[-1][tag] = []


        data[-1][tag].append(token)

    # reassemble the output into a list of dicts.
    output = [
        dict([(k, smartJoin(tokens)) for k, tokens in ingredient.items()])
        for ingredient in data
        if len(ingredient)
    ]

    # Add the raw ingredient phrase
    for i, v in enumerate(output):
        output[i]["input"] = smartJoin(
            [" ".join(tokens) for k, tokens in display[i]])

    return output[0]

format_ingredient_output(epi_ingredients["input"].values[0], epi_ingredients["predicted"].values[0])

{'qty': '2.50',
 'other': 'large',
 'name': 'garlic clove',
 'input': '2.50 large garlic clove'}

In [25]:
parsed = epi_ingredients.apply(lambda x: format_ingredient_output(x.input, x.predicted), axis=1)
parsed.index
epi_ingredients = pd.DataFrame(parsed.tolist(), index=parsed.index)
epi_ingredients.head()



IndexError: ('list index out of range', 'occurred at index http://www.epicurious.com/recipes/food/views/roast-leg-of-lamb-with-tarragon-mint-butter-352043')

In [ ]:
len(epi_ingredients.name.unique())

In [ ]:


epi_clone = epi_ingredients.copy()
c = Counter()
c2 = Counter()
c.update(epi_ingredients.name)

cols_to_drop = {k:v for k, v in c.items() if v < 5}
cols_to_keep = {k:v for k, v in c.items() if v >= 5}
for k1 in list(cols_to_drop.keys()):
    for k2, v2 in sorted(cols_to_keep.items(), key=operator.itemgetter(1)):
        if k2 == k2:
            #print(k1, k2)
            if k2 in k1:
                #print("k1: ", k1, "k2: ", k2)
                cols_to_keep[k2] += cols_to_drop[k1]
                #print(epi_clone[epi_clone.name == k1].name)
                epi_clone.loc[epi_clone.name == k1, "name"] = k2
                #print(epi_clone[epi_clone.name == k1].name)
                #print(epi_clone[epi_clone.name == k2].name)
                del cols_to_drop[k1]
                break
print(len(cols_to_drop))
#rows_to_drop = epi_ingredients[ epi_ingredients[cols_to_drop].sum(axis=1) > 0 ].index
#print(rows_to_drop)

In [ ]:
epi_clone.head()
epi_ingredients = epi_clone.copy()

In [ ]:
print(len(epi_ingredients.name.unique()))
#rows_to_drop = epi_ingredients.loc[epi_ingredients.name.isin(cols_to_drop)].index
epi_ingredients = epi_ingredients.drop(epi_ingredients.loc[epi_ingredients.name.isin(cols_to_drop)].index)
#print(rows_to_drop)
print(len(epi_ingredients.name.unique()))

In [ ]:
epi_ingredients.shape

In [ ]:
def qty2Decimal(qty):
    try:
        qty = float(qty)
    except ValueError:
        qty = np.nan

    return qty
epi_ingredients["qty"] = epi_ingredients["qty"].apply(lambda x: qty2Decimal(x))

In [ ]:
# Now we convert as many units as possible to metric
epi_ingredients.loc[epi_ingredients.unit == "pound", "qty"] *= 453.592
epi_ingredients.loc[epi_ingredients.unit == "pound", "unit"] = "grams"

epi_ingredients.loc[epi_ingredients.unit == "teaspoon", "qty"] *= 4.92892
epi_ingredients.loc[epi_ingredients.unit == "teaspoon", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "tablespoon", "qty"] *= 14.7868
epi_ingredients.loc[epi_ingredients.unit == "tablespoon", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "cup", "qty"] *= 236.588
epi_ingredients.loc[epi_ingredients.unit == "cup", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "pinch", "qty"] *= 4.92892 * (1 / 16)
epi_ingredients.loc[epi_ingredients.unit == "pinch", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "dash", "qty"] *= 4.92892 * (1 / 8)
epi_ingredients.loc[epi_ingredients.unit == "dash", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "ounce", "qty"] *= 28.3495
epi_ingredients.loc[epi_ingredients.unit == "ounce", "unit"] = "grams"

epi_ingredients.loc[epi_ingredients.unit == "fluid ounce", "qty"] *= 29.5735
epi_ingredients.loc[epi_ingredients.unit == "fluid ounce", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "pint", "qty"] *= 473.176
epi_ingredients.loc[epi_ingredients.unit == "pint", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "quart", "qty"] *= 946.353
epi_ingredients.loc[epi_ingredients.unit == "quart", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "liter", "qty"] *= 1000
epi_ingredients.loc[epi_ingredients.unit == "liter", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "gallon", "qty"] *= 3785.41
epi_ingredients.loc[epi_ingredients.unit == "gallon", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "drop", "qty"] *= 0.05
epi_ingredients.loc[epi_ingredients.unit == "drop", "unit"] = "milliliters"

epi_ingredients.loc[epi_ingredients.unit == "jigger", "qty"] *= 44.3603
epi_ingredients.loc[epi_ingredients.unit == "jigger", "unit"] = "milliliters"

epi_ingredients.head(10)

In [ ]:
epi_vec = epi_ingredients.pivot_table(
    index=epi_ingredients.index, columns="name", values="qty", aggfunc=np.mean
)
epi_vec = epi_vec.join(epi_data[["avg_rating", "best_rating", "worst_rating", "prepare_again_rating",
                                    "num_reviews", "total_time", "tags", "title"]])

epi_vec.fillna(0, inplace=True)
epi_vec.head()

In [ ]:
epi_vec.shape

In [ ]:
cols to drop = 
for col in epi_vec.columns:
    if len(epi_vec[epi_vec['" -thick Pullman bread'] > 0]) == 1:
        

In [ ]:
from collections import Counter
c = Counter()  
epi_vec["tags"].apply(lambda x: c.update(x))
print(c)

In [ ]:
plt.figure(figsize=(16, 10))
plt.matshow(epi_vec[columns].corr())
plt.show()

In [ ]:
# Let's save our dataframe so we can look at it without having to reload and recompute everything later.
# pickle is much faster than csv
recipe_df.to_pickle("../data/processed/recipe_vectors.pickle")

In [ ]:
recipe_df.columns

In [ ]:
recipe_df.loc[recipe_df['" bake pie crust'] > 0]

In [ ]:
# Here we compute the t-SNE
# WARNING: THIS TAKES A LONG TIME FOR LARGE DATAFRAMES
from sklearn.manifold import TSNE
columns = [x for x in epi_vec.columns if x not in ["avg_rating", "best_rating", "worst_rating", "prepare_again_rating",
                                    "num_reviews", "total_time", "tags", "title", "color"]]
data = epi_vec[columns].values
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=5000)
tsne_results = tsne.fit_transform(data)
print("tsne done")
epi_vec["tsne-2d-one"] = tsne_results[:, 0]
epi_vec["tsne-2d-two"] = tsne_results[:, 1]


In [ ]:
#recipe_df = pd.read_pickle("../data/processed/recipe_vectors.pickle")

In [ ]:
#from sklearn.manifold import TSNE
#recipe_df = recipe_df.iloc[0:2000]
#data = recipe_df[recipe_df.columns].values

In [ ]:
#tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=5000)
#tsne_results = tsne.fit_transform(data)
#print("tsne done")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 10))
#recipe_df["tsne-2d-one"] = tsne_results[:, 0]
#recipe_df["tsne-2d-two"] = tsne_results[:, 1]
#tsne_1 = tsne_results[:, 0]
epi_vec["color"] = epi_vec["tags"].apply(lambda x: "Dessert" if "Dessert" in x else "Not Dessert")
sns.scatterplot(x="tsne-2d-one", y="tsne-2d-two", data=epi_vec, hue="num_reviews", linewidth=0)
plt.show()

In [ ]:
## Here I'm just exploring outliers in the t-SNE
#sample_df = recipe_df.loc[recipe_df["tsne-2d-two"] >= 50]
#drop_cols = sample_df.columns[(sample_df == 0).sum() == sample_df.shape[0]]
#sample_df.drop(drop_cols, axis=1, inplace=True)
#sample_df.columns.unique().values
#sample_df.head(15)

In [ ]:
#sample_df.columns.unique().values